In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.2)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.66 sec - Loss 0.204314 - ACC 72.45% - ACC Mean 72.45% - AUC 77.76% - AUC Mean 77.76% - Deter 000
Ite 00050 - 0.86 sec - Loss 0.093729 - ACC 72.84% - ACC Mean 72.63% - AUC 75.73% - AUC Mean 76.93% - Deter 027
Ite 00073 - 0.88 sec - Loss 0.078966 - ACC 72.60% - ACC Mean 72.42% - AUC 77.17% - AUC Mean 76.44% - Deter 050
Early stop ite 73, rollback to correction of ite 23, whith acc of 73.35% and auc of 80.01%
OUR METHOD RUN 1/10 - acc: 73.35% +- 0.0% - auc: 80.01% +- 0.0%
Ite 00000 - 0.92 sec - Loss 0.215497 - ACC 73.89% - ACC Mean 73.89% - AUC 78.18% - AUC Mean 78.18% - Deter 000
Ite 00050 - 0.93 sec - Loss 0.107702 - ACC 73.13% - ACC Mean 72.81% - AUC 77.09% - AUC Mean 77.44% - Deter 040
Ite 00060 - 0.94 sec - Loss 0.098378 - ACC 71.94% - ACC Mean 72.82% - AUC 78.57% - AUC Mean 77.43% - Deter 050
Early stop ite 60, rollback to correction of ite 10, whith acc of 72.3% and auc of 78.79%
OUR METHOD RUN 2/10 - acc: 72.3% +- 0.0% - auc: 78.79% +- 0.0%
Ite 00000 - 1.02 sec - Los

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 71.71% +- 0.0% - auc: 76.48% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 51.47% +- 2.8% - auc: 56.1% +- 3.07%
SFIL RUN 2/10 - acc: 50.71% +- 1.5% - auc: 53.59% +- 2.21%
SFIL RUN 3/10 - acc: 50.89% +- 1.78% - auc: 57.43% +- 3.89%
SFIL RUN 4/10 - acc: 51.59% +- 2.1% - auc: 56.91% +- 3.28%
SFIL RUN 5/10 - acc: 53.06% +- 2.52% - auc: 54.77% +- 1.29%
SFIL RUN 6/10 - acc: 53.47% +- 2.99% - auc: 57.3% +- 4.38%
SFIL RUN 7/10 - acc: 51.1% +- 1.48% - auc: 56.3% +- 3.89%
SFIL RUN 8/10 - acc: 52.3% +- 2.83% - auc: 56.35% +- 2.98%
SFIL RUN 9/10 - acc: 50.0% +- 0.0% - auc: 57.73% +- 2.81%
SFIL RUN 10/10 - acc: 52.45% +- 3.74% - auc: 56.55% +- 3.56%
SFIL GLOBAL - acc: 51.7% +- 1.04% - auc: 56.3% +- 1.2%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 70.88% +- 3.84% - auc: 71.87% +- 5.01%
PFIL RUN 2/10 - acc: 71.07% +- 2.47% - auc: 76.12% +- 1.7%
PFIL RUN 3/10 - acc: 70.58% +- 4.29% - auc: 74.43% +- 4.95%
PFIL RUN 4/10 - acc: 69.65% +- 3.14% - auc: 74.81% +- 6.21%
PFIL RUN 5/10 - acc: 70.3% +- 1.76% - auc: 74.11% +- 2.89%
PFIL RUN 6/10 - acc: 70.15% +- 3.02% - auc: 72.86% +- 5.91%
PFIL RUN 7/10 - acc: 70.22% +- 1.79% - auc: 75.13% +- 3.13%
PFIL RUN 8/10 - acc: 70.22% +- 4.53% - auc: 74.15% +- 4.63%
PFIL RUN 9/10 - acc: 69.78% +- 4.03% - auc: 75.27% +- 3.61%
PFIL RUN 10/10 - acc: 70.13% +- 2.49% - auc: 74.66% +- 3.61%
PFIL GLOBAL - acc: 70.3% +- 0.42% - auc: 74.34% +- 1.16%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 52.61% +- 3.71% - auc: 56.64% +- 4.85%
SPOL RUN 2/10 - acc: 52.74% +- 2.56% - auc: 56.11% +- 4.02%
SPOL RUN 3/10 - acc: 57.39% +- 4.96% - auc: 61.14% +- 2.82%
SPOL RUN 4/10 - acc: 53.81% +- 3.05% - auc: 59.9% +- 2.76%
SPOL RUN 5/10 - acc: 52.55% +- 2.51% - auc: 56.47% +- 2.17%
SPOL RUN 6/10 - acc: 53.75% +- 2.55% - auc: 57.03% +- 3.55%
SPOL RUN 7/10 - acc: 51.87% +- 2.42% - auc: 56.12% +- 3.15%
SPOL RUN 8/10 - acc: 54.66% +- 4.66% - auc: 56.46% +- 3.18%
SPOL RUN 9/10 - acc: 54.83% +- 3.54% - auc: 59.04% +- 4.2%
SPOL RUN 10/10 - acc: 55.88% +- 4.36% - auc: 58.04% +- 4.98%
SPOL GLOBAL - acc: 54.01% +- 1.63% - auc: 57.69% +- 1.68%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 68.85% +- 4.65% - auc: 74.34% +- 6.06%
PPOL RUN 2/10 - acc: 70.31% +- 5.39% - auc: 78.06% +- 5.05%
PPOL RUN 3/10 - acc: 70.79% +- 3.11% - auc: 73.56% +- 2.87%
PPOL RUN 4/10 - acc: 69.27% +- 6.67% - auc: 75.62% +- 6.62%
PPOL RUN 5/10 - acc: 71.72% +- 2.23% - auc: 76.01% +- 3.76%
PPOL RUN 6/10 - acc: 69.21% +- 2.38% - auc: 75.88% +- 2.79%
PPOL RUN 7/10 - acc: 70.4% +- 4.56% - auc: 75.27% +- 6.03%
PPOL RUN 8/10 - acc: 69.65% +- 4.56% - auc: 75.59% +- 5.03%
PPOL RUN 9/10 - acc: 69.94% +- 4.64% - auc: 74.58% +- 4.38%
PPOL RUN 10/10 - acc: 68.53% +- 2.63% - auc: 71.01% +- 3.57%
PPOL GLOBAL - acc: 69.87% +- 0.92% - auc: 74.99% +- 1.75%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
